## Check crashes in small clusters

Get crashes in smaller clusters.

In [1]:
import pandas as pd
import utils.config as config
import utils.util as util
import numpy as np

df_err_grouped_k = pd.read_excel(config.path_default.joinpath('Clustering/clusters_Kaggle.xlsx'))
df_err_grouped_g = pd.read_excel(config.path_default.joinpath('Clustering/clusters_GitHub.xlsx'))

path_default=WindowsPath('C:/Users/yirwa29/Downloads/data_jupyter_nbs_empirical')


In [2]:
selected_clusters_g = pd.read_excel(config.path_default.joinpath('Sampling/cluster_size_samples_g.xlsx'))
selected_clusters_k = pd.read_excel(config.path_default.joinpath('Sampling/cluster_size_samples_k.xlsx'))

# big clusters (sample size >= 5)
clusters_big_k = selected_clusters_k[selected_clusters_k.sample_size>=5].cluster_id[1:]
clusters_big_g = selected_clusters_g[selected_clusters_g.sample_size>=5].cluster_id[1:]

In [3]:
# get crashes from small clusters (sample size < 5)
df_err_grouped_k_small = df_err_grouped_k[~df_err_grouped_k.pregroup_cluster.isin(clusters_big_k)]
df_err_grouped_g_small = df_err_grouped_g[~df_err_grouped_g.pregroup_cluster.isin(clusters_big_g)]

In [4]:
# randomly select 10 small clusters from each source
sample_k_small_clusters = np.random.choice(df_err_grouped_k_small.pregroup_cluster.unique(), size=10, replace=False)
sample_g_small_clusters = np.random.choice(df_err_grouped_g_small.pregroup_cluster.unique(), size=10, replace=False)

In [5]:
# randomly select min(5,cluster size) crashes from each cluster
# Kaggle
for j in range(len(sample_k_small_clusters)):
    skc = sample_k_small_clusters[j]
    print('\033[1m'+"+++++{}/{}: Error examples for Cluster {} in Kaggle error notebooks.+++++\n".format(j+1, 10, skc)+'\033[0m')
    df_target = df_err_grouped_k_small[df_err_grouped_k_small.pregroup_cluster==skc]
    n = min(5, len(df_target))
    tmp = df_target.sample(n=n, random_state=10)
    for i in range(len(tmp)):
        print("-----Error example {}/{} with eid {}.-----\n".format(i+1, n, tmp.iloc[i].eid))
        util.print_traceback(tmp.iloc[i].traceback)
        print("---------------------------------------------------------------------------\n")
    print("+++++Error examples for Cluster {} in Kaggle error notebooks end.+++++\n".format(skc))

+++++1/10: Error examples for Cluster 7167 in Kaggle error notebooks.+++++

-----Error example 1/1 with eid 35f8aa8f-f76a-3505-8641-8aad4332d21c.-----

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
Cell In[76], line 30
     21 param_grid = {
     22     'lstm__units': [10, 20, 30],
     23     'lstm__dropout': [0.2, 0.3, 0.4],
   (...)
     26     'batch_size': [32, 64]
     27 }
     29 # Call the function to find the best parameters
---> 30 best_params, best_model = find_best_lstm_parameters(X_train, y_train, param_grid)
     32 # Print the best parameters
     33 print("Best Parameters:")

Cell In[76], line 11, in find_best_lstm_parameters(X_train, y_train, param_grid)
      8 lstm.compile(loss=huber_loss, optimizer='adam')
     10 grid_search = GridSearchCV(estimator=lstm, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3)
---> 11 grid_search.fit(X_train, y_train)
 

In [6]:
# GitHub
for j in range(len(sample_g_small_clusters)):
    sgc = sample_g_small_clusters[j]
    print('\033[1m'+"+++++{}/{}: Error examples for Cluster {} in GitHub error notebooks.+++++\n".format(j+1, 10, sgc)+'\033[0m')
    df_target = df_err_grouped_g_small[df_err_grouped_g_small.pregroup_cluster==sgc]
    n = min(5, len(df_target))
    tmp = df_target.sample(n=n, random_state=30)
    for i in range(len(tmp)):
        print("-----Error example {}/{} with eid {}.-----\n".format(i+1, n, tmp.iloc[i].eid))
        util.print_traceback(tmp.iloc[i].traceback)
        print("---------------------------------------------------------------------------\n")
    print("+++++Error examples for Cluster {} in GitHub error notebooks end.+++++\n".format(sgc))

+++++1/10: Error examples for Cluster 5563 in GitHub error notebooks.+++++

-----Error example 1/1 with eid 23475959-9147-358d-a8e9-c546adf5caa0.-----

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
D:\Users\zanny\Anaconda3\lib\site-packages\gmaps\geotraitlets.py in _validate_latitude(latitude)
    298     try:
--> 299         latitude = float(latitude)
    300     except (TypeError, ValueError):

ValueError: could not convert string to float: 

During handling of the above exception, another exception occurred:

TraitError                                Traceback (most recent call last)
D:\Users\zanny\Anaconda3\lib\site-packages\traitlets\traitlets.py in validate_elements(self, obj, value)
   2443             try:
-> 2444                 v = t._validate(obj, v)
   2445             except TraitError:

D:\Users\zanny\Anaconda3\lib\site-packages\traitlets\traitlets.py in _validate(sel

Potentially interesting clusters:

Round 1:
Kaggle:

        ○ Cluster 7348 (size 4): "ValueError: Cannot assign value to variable ' embedding_2/embeddings:0': Shape mismatch.The variable shape (514157, 300), and the assigned value shape (514157,) are incompatible." shape mismatch during model construction
        ○ Cluster 7274 (size 2): "InternalError: RET_CHECK failure (tensorflow/core/tpu/graph_rewrite/distributed_tpu_rewrite_pass.cc:2008) arg_shape.handle_type != DT_INVALID  input edge: [id=881 sequential_keras_layer_2728:0 -> cluster_train_function:63] [Op:__inference_train_function_3219]" due to (probably) improper training dataset size
        ○ Cluster 7345 (size 4): "RuntimeError: stack expects each tensor to be equal size, but got [19] at entry 0 and [21] at entry 1"
	
GitHub:

        ○ Cluster 251 (size 13): "RuntimeError: Calculated padded input size per channel: (1 x 1). Kernel size: (3 x 3). Kernel size can't be greater than actual input size" (torch)
        ○ Cluster 7037 (size 25): "ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 3 dimension(s) and the array at index 1 has 2 dimension(s)" (numpy)
        
Round 2:
Kaggle:

        ○ (In the sample) Cluster 720 (size 37): "ValueError: Input 0 of layer "sequential_1" is incompatible with the layer: expected shape=(None, 32), found shape=(None, 38)"
        ○ (In the sample) Cluster 6655 (size 5): "AssertionError: Torch not compiled with CUDA enabled"
        ○ Cluster 7024 (size 1): "ValueError: Unable to load weights saved in HDF5 format into a subclassed Model which has not created its variables yet. Call the Model first, then load the weights." initialization error
        
GitHub:

        ○ Cluster 7307 (size 63): "ValueError: Number of features of the model must match the input. Model n_features is 11 and input n_features is 10 " tensorshape mismatch in model construction
        ○ (In the sample) Cluster 7314 (size 21): "RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU."

In [8]:
# check if any of them are included in the samples
df_mlerr_labels = pd.read_excel(config.path_default.joinpath('Manual_labeing/cluster_sampled_labeled.xlsx'),
                                sheet_name = "Del-All",
                                keep_default_na=False)

C:\Users\yirwa29\AppData\Local\anaconda3\envs\chatgpt\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [9]:
for i in [7348,7274,7345,251,7037]:
    print("{} cluster size".format(i))
    print("    Kaggle: {}".format(sum(df_err_grouped_k.pregroup_cluster==i)))
    print("    GitHub: {}".format(sum(df_err_grouped_g.pregroup_cluster==i)))
    print("    If being sampled: {}".format(sum(df_mlerr_labels.pregroup_cluster==i)>0))
#     print("\n")

7348 cluster size
    Kaggle: 4
    GitHub: 1
    If being sampled: False
7274 cluster size
    Kaggle: 2
    GitHub: 0
    If being sampled: False
7345 cluster size
    Kaggle: 4
    GitHub: 12
    If being sampled: False
251 cluster size
    Kaggle: 0
    GitHub: 13
    If being sampled: False
7037 cluster size
    Kaggle: 1
    GitHub: 25
    If being sampled: False


In [10]:
for i in [720,6655,7024,7307,7314]:
    print("{} cluster size".format(i))
    print("    Kaggle: {}".format(sum(df_err_grouped_k.pregroup_cluster==i)))
    print("    GitHub: {}".format(sum(df_err_grouped_g.pregroup_cluster==i)))
    print("    If being sampled: {}".format(sum(df_mlerr_labels.pregroup_cluster==i)>0))
#     print("\n")

720 cluster size
    Kaggle: 37
    GitHub: 155
    If being sampled: True
6655 cluster size
    Kaggle: 5
    GitHub: 82
    If being sampled: True
7024 cluster size
    Kaggle: 1
    GitHub: 3
    If being sampled: False
7307 cluster size
    Kaggle: 2
    GitHub: 63
    If being sampled: False
7314 cluster size
    Kaggle: 5
    GitHub: 21
    If being sampled: True
